In [1]:
import json
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from pyvi import ViTokenizer


In [2]:

# ==============================================================================
# 1. LOAD DỮ LIỆU & CHIA TẬP (80 - 10 - 10)
# ==============================================================================
input_file = 'train_bio.json'

print(f"--- 1. LOADING DATA ---")
try:
    with open(input_file, 'r', encoding='utf-8') as f:
        all_sents = json.load(f)
    print(f"Tổng số câu: {len(all_sents)}")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy {input_file}")
    exit()

# Chia Train (80%) và Temp (20%)
train_sents, temp_sents = train_test_split(all_sents, test_size=0.2, random_state=42)
# Chia Temp thành Val (10% tổng) và Test (10% tổng) -> Chia đôi temp_sents
val_sents, test_sents = train_test_split(temp_sents, test_size=0.5, random_state=42)

print(f"Train size: {len(train_sents)} (80%)")
print(f"Val size:   {len(val_sents)} (10%)")
print(f"Test size:  {len(test_sents)} (10%)")


--- 1. LOADING DATA ---
Tổng số câu: 1136
Train size: 908 (80%)
Val size:   114 (10%)
Test size:  114 (10%)


In [16]:
import pickle
import os

In [ ]:

# ==============================================================================
# 2. FEATURE ENGINEERING (CHO CLASSICAL ML)
# ==============================================================================
def word2features(sent, i):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.has_underscore': '_' in word,
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({'-1:word.lower()': word1.lower(), '-1:word.istitle()': word1.istitle()})
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({'+1:word.lower()': word1.lower(), '+1:word.istitle()': word1.istitle()})
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

print("\n--- 2. PREPARING FEATURES ---")
# Tạo dữ liệu cho các mô hình truyền thống
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]
# Lưu ý: LR và RF không dùng chuỗi, nên ta không cần quan tâm thứ tự câu trong validation cho việc training cơ bản




--- 2. PREPARING FEATURES ---
Saved: crf_model.pkl


In [19]:

# ==============================================================================
# 3. HUẤN LUYỆN & ĐÁNH GIÁ: CRF
# ==============================================================================
print("\n--- 3. TRAINING CRF ---")
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)
crf.fit(X_train, y_train)

print(">>> REPORT CRF (trên tập Test):")
y_pred_crf = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred_crf, digits=4))
save_dir = 'saved_models'
os.makedirs(save_dir, exist_ok=True)
with open(os.path.join(save_dir, 'crf_model.pkl'), 'wb') as f:
    pickle.dump(crf, f)
print("Saved: crf_model.pkl")



--- 3. TRAINING CRF ---
>>> REPORT CRF (trên tập Test):
               precision    recall  f1-score   support

       B-CHAR     0.8684    0.8462    0.8571        39
        B-LOC     0.6667    0.6479    0.6571        71
        B-ORG     0.7846    0.7083    0.7445        72
        B-PER     0.8169    0.8529    0.8345        68
B-TIME / DATE     0.3704    0.5882    0.4545        17
  B-TIME/DATE     0.6000    0.5085    0.5505        59
       B-WORK     0.5833    0.2333    0.3333        30
       I-CHAR     0.9286    0.5000    0.6500        26
        I-LOC     0.4667    0.3889    0.4242        18
        I-ORG     0.8480    0.7737    0.8092       137
        I-PER     0.8000    0.9524    0.8696        21
I-TIME / DATE     0.3333    0.4444    0.3810         9
  I-TIME/DATE     0.7215    0.6786    0.6994        84
       I-WORK     0.6389    0.3108    0.4182        74
            O     0.9178    0.9658    0.9412      1665

     accuracy                         0.8674      2390
    ma

In [20]:


# ==============================================================================
# 4. HUẤN LUYỆN & ĐÁNH GIÁ: LOGISTIC REGRESSION & RANDOM FOREST
# ==============================================================================
print("\n--- 4. TRAINING LR & RF ---")
# Flatten dữ liệu (Chuyển từ list of lists sang list phẳng)
X_train_flat = [item for sublist in X_train for item in sublist]
y_train_flat = [item for sublist in y_train for item in sublist]

X_test_flat = [item for sublist in X_test for item in sublist]
y_test_flat = [item for sublist in y_test for item in sublist]

# Vector hóa
v = DictVectorizer(sparse=False)
X_train_vec = v.fit_transform(X_train_flat)
X_test_vec = v.transform(X_test_flat) # Chỉ transform, không fit lại

# --- Logistic Regression ---
lr = LogisticRegression(max_iter=500, multi_class='ovr', n_jobs=-1)
lr.fit(X_train_vec, y_train_flat)
print(">>> REPORT Logistic Regression:")
print(classification_report(y_test_flat, lr.predict(X_test_vec), digits=4))

# --- Random Forest ---
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)
rf.fit(X_train_vec, y_train_flat)
print(">>> REPORT Random Forest:")
print(classification_report(y_test_flat, rf.predict(X_test_vec), digits=4))

with open(os.path.join(save_dir, 'vectorizer.pkl'), 'wb') as f:
    pickle.dump(v, f)
print("Saved: vectorizer.pkl")

with open(os.path.join(save_dir, 'lr_model.pkl'), 'wb') as f:
    pickle.dump(lr, f)
print("Saved: lr_model.pkl")

with open(os.path.join(save_dir, 'rf_model.pkl'), 'wb') as f:
    pickle.dump(rf, f)
print("Saved: rf_model.pkl")


--- 4. TRAINING LR & RF ---


c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


>>> REPORT Logistic Regression:
               precision    recall  f1-score   support

       B-CHAR     0.8056    0.7436    0.7733        39
        B-LOC     0.7015    0.6620    0.6812        71
        B-ORG     0.7500    0.6250    0.6818        72
        B-PER     0.8108    0.8824    0.8451        68
B-TIME / DATE     0.3846    0.2941    0.3333        17
  B-TIME/DATE     0.4783    0.3729    0.4190        59
       B-WORK     0.7273    0.2667    0.3902        30
       I-CHAR     0.6429    0.3462    0.4500        26
        I-LOC     0.6250    0.2778    0.3846        18
        I-ORG     0.8352    0.5547    0.6667       137
        I-PER     0.7407    0.9524    0.8333        21
I-TIME / DATE     0.6667    0.2222    0.3333         9
  I-TIME/DATE     0.6667    0.6190    0.6420        84
       I-WORK     0.8000    0.1622    0.2697        74
            O     0.8798    0.9760    0.9254      1665

     accuracy                         0.8439      2390
    macro avg     0.7010    0.5

c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri

In [21]:


# ==============================================================================
# 5. HUẤN LUYỆN & ĐÁNH GIÁ: BI-LSTM
# ==============================================================================
print("\n--- 5. TRAINING BI-LSTM ---")

# Tạo từ điển từ TOÀN BỘ dữ liệu để tránh lỗi out-of-vocab
words = list(set([t[0] for sent in all_sents for t in sent]))
tags = list(set([t[1] for sent in all_sents for t in sent]))

if "UNK" not in words: words.append("UNK")
if "PAD" not in words: words.append("PAD")
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}

MAX_LEN = 50
n_words = len(words)
n_tags = len(tags)

# Hàm helper để encode dữ liệu cho DL
def encode_data(sents, word2idx, tag2idx, max_len):
    X = [[word2idx.get(w[0], word2idx["UNK"]) for w in s] for s in sents]
    X = pad_sequences(X, maxlen=max_len, padding="post", value=word2idx["PAD"])
    
    y = [[tag2idx[w[1]] for w in s] for s in sents]
    y = pad_sequences(y, maxlen=max_len, padding="post", value=tag2idx["O"]) # Pad label bằng 'O'
    y = [to_categorical(i, num_classes=n_tags) for i in y]
    return X, np.array(y)

# Encode các tập
X_train_dl, y_train_dl = encode_data(train_sents, word2idx, tag2idx, MAX_LEN)
X_val_dl, y_val_dl = encode_data(val_sents, word2idx, tag2idx, MAX_LEN) # Dùng Validation ở đây
X_test_dl, y_test_dl = encode_data(test_sents, word2idx, tag2idx, MAX_LEN)

# Build Model
model = Sequential([
    Embedding(input_dim=n_words, output_dim=50, input_length=MAX_LEN),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    Dropout(0.3),
    TimeDistributed(Dense(n_tags, activation="softmax"))
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train có sử dụng validation_data
history = model.fit(X_train_dl, y_train_dl, 
                    validation_data=(X_val_dl, y_val_dl), 
                    batch_size=32, epochs=50, verbose=1)

# Đánh giá Bi-LSTM
print(">>> REPORT Bi-LSTM:")
y_pred_probs = model.predict(X_test_dl)
y_pred_indices = np.argmax(y_pred_probs, axis=-1)
y_test_indices = np.argmax(y_test_dl, axis=-1)

# Chuyển ngược từ số về nhãn (bỏ padding để tính chính xác)
pred_tags_flat = []
true_tags_flat = []

for i in range(len(test_sents)):
    # Lấy độ dài thực tế của câu (tránh lấy phần padding)
    true_len = len(test_sents[i])
    # Chỉ lấy phần dự đoán tương ứng với độ dài câu
    p_tags = [idx2tag[idx] for idx in y_pred_indices[i][:true_len]]
    t_tags = [idx2tag[idx] for idx in y_test_indices[i][:true_len]]
    
    pred_tags_flat.extend(p_tags)
    true_tags_flat.extend(t_tags)

print(classification_report(true_tags_flat, pred_tags_flat, digits=4))

bi_lstm_config = {
    'word2idx': word2idx,
    'tag2idx': tag2idx,
    'idx2tag': idx2tag,
    'max_len': MAX_LEN
}

with open(os.path.join(save_dir, 'bi_lstm_config.pkl'), 'wb') as f:
    pickle.dump(bi_lstm_config, f)
print("Saved: bi_lstm_config.pkl")

model.save(os.path.join(save_dir, 'bi_lstm_model.keras'))
print("Saved: bi_lstm_model.keras")



--- 5. TRAINING BI-LSTM ---


c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.8380 - loss: 1.2233 - val_accuracy: 0.8768 - val_loss: 0.5889
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.8706 - loss: 0.5930 - val_accuracy: 0.8768 - val_loss: 0.5416
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8706 - loss: 0.5614 - val_accuracy: 0.8768 - val_loss: 0.5216
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.8706 - loss: 0.5298 - val_accuracy: 0.8768 - val_loss: 0.4932
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.8720 - loss: 0.4939 - val_accuracy: 0.8791 - val_loss: 0.4613
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.8779 - loss: 0.4552 - val_accuracy: 0.8851 - val_loss: 0.4329
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.8857 - loss: 0.4175 - val_accuracy: 0.8912 - val_loss: 0.4071
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8928 - loss: 0.3830 - val_accuracy: 0.8972 - v

c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\vthuy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri

Saved: bi_lstm_model.keras


In [23]:
import pickle
import re
import numpy as np
import os
from pyvi import ViTokenizer
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# ==============================================================================
# 1. HÀM TIỀN XỬ LÝ (GIỐNG HỆT LÚC TRAIN)
# ==============================================================================
def preprocess_text_for_prediction(text):
    """
    Mô phỏng lại hàm 'convert_to_bio_with_cleaning' nhưng bỏ phần gán nhãn.
    Input: Câu văn bản thô.
    Output: List các token đã được làm sạch y hệt dữ liệu train.
    """
    # 1. Tokenize bằng PyVi
    tokenized_text = ViTokenizer.tokenize(text)
    raw_tokens = tokenized_text.split()
    
    cleaned_tokens = []
    
    # 2. Làm sạch bằng Regex (COPY Y HỆT TỪ CODE CỦA BẠN)
    for token in raw_tokens:
        clean_t = re.sub(r'[^\w\s\d_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂÂÊÔƠƯưăâêôơư]', '', token)
        
        # Chỉ giữ lại token nào còn nội dung sau khi clean
        if clean_t:
            cleaned_tokens.append(clean_t)
            
    return cleaned_tokens

# ==============================================================================
# 2. HÀM TẠO FEATURES (GIỐNG LÚC TRAIN)
# ==============================================================================
def get_features_for_prediction(sent):
    sent_features = []
    for i in range(len(sent)):
        word = sent[i]
        features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'word.has_underscore': '_' in word, # Feature quan trọng của PyVi
        }
        if i > 0:
            word1 = sent[i-1]
            features.update({'-1:word.lower()': word1.lower(), '-1:word.istitle()': word1.istitle()})
        else:
            features['BOS'] = True

        if i < len(sent)-1:
            word1 = sent[i+1]
            features.update({'+1:word.lower()': word1.lower(), '+1:word.istitle()': word1.istitle()})
        else:
            features['EOS'] = True
        sent_features.append(features)
    return sent_features

# ==============================================================================
# 3. LOAD MODEL
# ==============================================================================
save_dir = 'saved_models' # Đảm bảo đường dẫn đúng

print("--- Loading Models ---")
# Load CRF
with open(os.path.join(save_dir, 'crf_model.pkl'), 'rb') as f:
    loaded_crf = pickle.load(f)

# Load Classical ML Components
with open(os.path.join(save_dir, 'vectorizer.pkl'), 'rb') as f:
    loaded_v = pickle.load(f)
with open(os.path.join(save_dir, 'lr_model.pkl'), 'rb') as f:
    loaded_lr = pickle.load(f)
with open(os.path.join(save_dir, 'rf_model.pkl'), 'rb') as f:
    loaded_rf = pickle.load(f)

# Load Bi-LSTM Components
with open(os.path.join(save_dir, 'bi_lstm_config.pkl'), 'rb') as f:
    lstm_config = pickle.load(f)
loaded_lstm = load_model(os.path.join(save_dir, 'bi_lstm_model.keras'))

word2idx = lstm_config['word2idx']
idx2tag = lstm_config['idx2tag']
MAX_LEN = lstm_config['max_len']
print(">>> Ready to predict!")

# ==============================================================================
# 4. THỰC HIỆN DỰ ĐOÁN
# ==============================================================================

# Dữ liệu mới (chưa từng thấy lúc train)
test_sentences = [
    'Tác phẩm "Cho tôi xin một vé đi tuổi thơ" của Nguyễn Nhật Ánh ra mắt năm 2008.',
    'Chí Phèo và Thị Nở là hai nhân vật kinh điển của nhà văn Nam Cao.',
    'Bình Ngô đại cáo do Nguyễn Trãi soạn thảo để tuyên cáo chiến thắng quân Minh.'
]

print("\n================ PREDICTION REPORT ================")

for text in test_sentences:
    print(f"\n📝 Input Raw: {text}")
    
    # BƯỚC QUAN TRỌNG: Dùng hàm preprocess chuẩn
    tokens = preprocess_text_for_prediction(text)
    print(f"🔧 Cleaned Tokens: {tokens}") 
    
    if not tokens:
        print("-> Câu này bị lọc hết sạch ký tự đặc biệt, bỏ qua.")
        continue

    # --- A. CRF ---
    features = get_features_for_prediction(tokens)
    pred_crf = loaded_crf.predict_single(features)
    
    # --- B. Logistic Regression & Random Forest ---
    features_vec = loaded_v.transform(features)
    pred_lr = loaded_lr.predict(features_vec)
    pred_rf = loaded_rf.predict(features_vec)
    
    # --- C. Bi-LSTM ---
    seq_idx = [word2idx.get(w, word2idx['UNK']) for w in tokens]
    seq_padded = pad_sequences([seq_idx], maxlen=MAX_LEN, padding='post', value=word2idx['PAD'])
    pred_prob_lstm = loaded_lstm.predict(seq_padded, verbose=0)
    pred_idx_lstm = np.argmax(pred_prob_lstm, axis=-1)[0]
    
    # Lấy nhãn tương ứng độ dài thực tế
    pred_lstm = [idx2tag[i] for i in pred_idx_lstm[:len(tokens)]]

    # --- IN KẾT QUẢ ---
    print(f"{'TOKEN':<20} | {'CRF':<10} | {'LogReg':<10} | {'Bi-LSTM':<10}")
    print("-" * 60)
    
    for i in range(len(tokens)):
        t_crf = pred_crf[i]
        t_lr = pred_lr[i]
        # t_rf = pred_rf[i] # Có thể thêm vào nếu muốn
        t_lstm = pred_lstm[i] if i < len(pred_lstm) else "O"
        
        # Chỉ in những token có nhãn khác 'O' để dễ nhìn (hoặc in hết tùy bạn)
        # if t_crf != 'O' or t_lstm != 'O': 
        print(f"{tokens[i]:<20} | {t_crf:<10} | {t_lr:<10} | {t_lstm:<10}")

--- Loading Models ---
>>> Ready to predict!

================ PREDICTION REPORT ================

📝 Input Raw: Tác phẩm "Cho tôi xin một vé đi tuổi thơ" của Nguyễn Nhật Ánh ra mắt năm 2008.
🔧 Cleaned Tokens: ['Tác_phẩm', 'Cho', 'tôi', 'xin', 'một', 'vé', 'đi', 'tuổi_thơ', 'của', 'Nguyễn_Nhật_Ánh', 'ra_mắt', 'năm', '2008']
TOKEN                | CRF        | LogReg     | Bi-LSTM   
------------------------------------------------------------
Tác_phẩm             | O          | O          | O         
Cho                  | O          | O          | O         
tôi                  | O          | O          | O         
xin                  | O          | O          | O         
một                  | O          | O          | O         
vé                   | O          | O          | O         
đi                   | O          | O          | O         
tuổi_thơ             | O          | O          | O         
của                  | O          | O          | O         
Nguyễn_Nhật_Án